In [ ]:
# Regret Heuristics
As 

In [ ]:
import math
import random
import matplotlib.pyplot as plt

random.seed(123)
plt.style.use("fivethirtyeight")

cities = [(random.random(), random.random()) for _ in range(30)]

distance_matrix = [
    [
        math.sqrt((start_city[0] - end_city[0])**2 + (start_city[1] - end_city[1])**2)
        for end_city in cities
    ]
    for start_city in cities
]

plt.scatter(
    [city[0] for city in cities],
    [city[1] for city in cities],
)
plt.axis("off")

In [ ]:
def regretful_insert(distance_matrix, start, k=2):
    unvisited_cities = set(range(len(distance_matrix))) - {start}
    tour = [start, start]
    total_length = 0

    while unvisited_cities: # O(n)
        regrets = []
        for city in unvisited_cities: # O(n)
            insertion_data = sorted(
                (
                    (
                        total_length
                        - distance_matrix[tour[i - 1]][tour[i]]
                        + distance_matrix[tour[i - 1]][city]
                        + distance_matrix[city][tour[i]],
                        i,
                    )
                    for i in range(1, len(tour)) 
                ),
                key=lambda x: x[0],
            ) # for the city, we have a sorted list of insertions, O(n log(n))
            if len(insertion_data) >= k:
                regret = insertion_data[k - 1][0] - insertion_data[0][0] # best length minus k-th best length
            else:
                regret = -insertion_data[0][0] # we don't have enough alternatives, so be greedy here

            length, best_i = insertion_data[0]

            regrets.append((regret, city, best_i, length))

        _, best_city, best_i, total_length = max(regrets, key=lambda x: x[0]) # pick city with highest regret, O(n)
        tour = tour[:best_i] + [best_city] + tour[best_i:]
        unvisited_cities.remove(best_city)

    return {"tour": tour, "length": total_length}

In [ ]:
def regretful_insert_with_best_start(distance_matrix, k):
    return min(
        (regretful_insert(distance_matrix, start, k) for start in range(len(distance_matrix))),
        key=lambda x: x["length"]
    )

In [ ]:
def plot(result, cities):
    plt.scatter(
        [city[0] for city in cities],
        [city[1] for city in cities],
    )

    plt.plot(
        [cities[result["tour"][i]][0] for i in range(len(result["tour"]))],
        [cities[result["tour"][i]][1] for i in range(len(result["tour"]))],
        c="k", linewidth=1, zorder=-1
    )

    plt.title(f"Length = {result['length']:.2f}")

    plt.axis("off")


result = regretful_insert_with_best_start(distance_matrix, k=2)
plot(result, cities)